In [1]:
import orjson

In [2]:
with open('eval_results/gorilla-openfunctions-v2-5.0bpw-h6-exl2-pretrained_score.json', 'rb') as f:
    results = orjson.loads(f.read())
with open('eval_outputs/gorilla-openfunctions-v2-5.0bpw-h6-exl2-pretrained.json', 'rb') as f:
    infer_output = orjson.loads(f.read())
with open('../../data/nlp.jsonl', 'rb') as f:
    label = [orjson.loads(line.strip()) for line in f if line.strip() != ""]

In [3]:
failed = {k: v for k, v in results.items() if v['target'] < 1 or v['tool'] < 1}
failed_heading = {k: v for k, v in results.items() if v['heading'] < 1}

In [8]:
# 230 Turret Foxtrot, engage the target at heading two seven five, red and yellow missile, deploy surface-to-air missiles. Turret Foxtrot, prepare for strike. It give [{'heading': '275', 'tool': 'red and yellow missile', 'target': 'deploy surface-to-air missiles'}]

In [4]:
failed_heading

{'54': {'heading': 0.0, 'target': 0, 'tool': 0},
 '68': {'heading': 0.0, 'target': 0, 'tool': 0},
 '133': {'heading': 0.0, 'target': 0, 'tool': 0},
 '247': {'heading': 0.0, 'target': 0, 'tool': 0},
 '330': {'heading': 0.0, 'target': 0, 'tool': 0},
 '359': {'heading': 0.0, 'target': 0, 'tool': 0},
 '511': {'heading': 0.0, 'target': 0, 'tool': 0},
 '773': {'heading': 0.0, 'target': 0, 'tool': 0},
 '788': {'heading': 0.0, 'target': 0, 'tool': 0},
 '949': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1117': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1174': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1254': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1257': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1266': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1268': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1282': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1337': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1361': {'heading': 0.0, 'target': 0, 'tool': 0},
 '1376': {'heading': 0.0, 'target': 0, 'too